We can get the data for game franchises from Giantbomb. My api key is f2577eac7f52b72170fc1e8c5f0cebb778f38155

In [ ]:
import requests
import json
import pandas as pd
import re
from pathlib import Path
import time
import os

## Acquiring and cleaning franchise IDs

Create the directory to store the raw info

In [ ]:
franchises_dir = Path('raw_json\\game_franchises\\franchises')
franchises_dir.mkdir(parents=True, exist_ok=True)

Get all the data from the franchise ids and save as json files

In [ ]:
user_agent = "Joshdharris"
headers = {'User-Agent': user_agent}

for x in range(0, 5550, 100):
    filename = "page{}-{}.json".format(x, x+99)
    if not Path(franchises_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchises/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=guid,name,description&offset={}".format(x)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(franchises_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)

This function will be used multiple times for the raw json files. Here we create an empty dataframe, then open all the json files in a specified directory and add them to the dataframe, before returning the dataframe

In [327]:
def create_dataframe(desired_name, raw_directory, add_path):
    #Create empty dataframe
    desired_name = pd.DataFrame(data=[], columns=None)

    #This line creates a list of all the json files in the directory
    files = list(Path(raw_directory).glob('*.json'))
    
    #Iterate through all json files in directory and add to dataframe
    for file in files:
        with open(file) as filein:
            tempDF = pd.read_json(filein)
            if add_path == True:
                tempDF["path"] = str(file)
            desired_name = pd.concat([desired_name, tempDF])       
    return desired_name

Collate all the downloaded files into a single dataframe

In [222]:
franchise_ids = create_dataframe("franchise_ids", franchises_dir, False)
franchise_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version
0,OK,100,0,100,5524,1,{'description': '<h2>Navigation</h2><h3>Sub-Fr...,1
1,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><figure data...,1
2,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>Metroid i...,1
3,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>Developed...,1
4,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>The Metal...,1
...,...,...,...,...,...,...,...,...
95,OK,100,900,100,5524,1,{'description': '<p>Imperium Galactica is semi...,1
96,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1030', 'na...",1
97,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1031', 'na...",1
98,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1032', 'na...",1


Remove unnecessary rows

Again, create function as it's used multiple times

In [220]:
def drop_unnecessary_columns(dataframe):
    dataframe = dataframe.drop(["error","limit", "offset", "number_of_page_results", "number_of_total_results", "status_code", "version"], axis = 1)
    return dataframe

In [224]:
franchise_ids = drop_unnecessary_columns(franchise_ids)
franchise_ids

,results
0,{'description': '<h2>Navigation</h2><h3>Sub-Fr...
1,{'description': '<h2>Overview</h2><figure data...
2,{'description': '<h2>Overview</h2><p>Metroid i...
3,{'description': '<h2>Overview</h2><p>Developed...
4,{'description': '<h2>Overview</h2><p>The Metal...
...,...
95,{'description': '<p>Imperium Galactica is semi...
96,"{'description': None, 'guid': '3025-1030', 'na..."
97,"{'description': None, 'guid': '3025-1031', 'na..."
98,"{'description': None, 'guid': '3025-1032', 'na..."


Extract information from results

In [225]:
franchise_ids["guid"] = [d.get('guid') for d in franchise_ids.results]
franchise_ids["name"] = [d.get('name') for d in franchise_ids.results]
franchise_ids["description"] = [d.get('description') for d in franchise_ids.results]

Get list of all franchise ids.

In [235]:
franchise_id_list = franchise_ids["guid"].tolist()
print(len(franchise_id_list))

5524


Make sure we have no duplicates of the ids

In [233]:
franchise_ids.guid.duplicated().sum()

0

## Acquiring game information for each franchise

Create directory for game franchises

In [ ]:
games_dir = Path('raw_json\\game_franchises\\games')
games_dir.mkdir(parents=True, exist_ok=True)

Download json files for each franchise. We have a cap of 200 requests an hour, hence we're using a time restriction within the loop to prevent the api from blocking us.

In [227]:
for id in franchise_id_list:
    filename = "{}.json".format(id)
    if not Path(games_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchise/{}/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=games".format(id)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(games_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)
        time.sleep(18)

Create a games dataframe from all the games in the directory

In [426]:
games_ids = create_dataframe("games_ids", games_dir, True)
games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,path
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-1.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-10.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-100.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-101.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-102.json
...,...,...,...,...,...,...,...,...,...
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-94.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-95.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-97.json
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\3025-98.json


Edit the path to be franchise ids and rename

In [427]:
games_ids["path"] = games_ids["path"].apply(lambda x : x[31:-5])
games_ids = games_ids.rename(columns = {"path":"franchise_id"})
games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,franchise_id
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-10
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-100
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-101
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-102
...,...,...,...,...,...,...,...,...,...
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-94
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-95
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-97
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-98


### Retrieving missing values

For some reason we have only 5341 results, but 5524 franchise ids. This is, we discovered, because the api is a little buggy. For some reason, having the field-list restriction on returns an empty result for the missing apis, so the workaround is to download all the information and extract the game information from that. 

In [418]:
# Lists all json files in directory
games_in_dir = os.listdir(games_dir)
# Removes .json extension from names
for i in range(len(games_in_dir)):
    games_in_dir[i] = games_in_dir[i][:-5]
# Creates a new list of all the games in the franchise list, but not in the directory
missing_apis = [x for x in franchise_id_list if x not in games_in_dir]
# Check number of missing files
print(len(missing_apis))

183


We'll put the missing files into a separate directory and make a separate dataframe so we don't mess up anything with the current dataframe

In [273]:
missing_games_dir = Path('raw_json\\game_franchises\\games\\missing')
missing_games_dir.mkdir(parents=True, exist_ok=True)

In [299]:
for id in missing_apis:
    filename = "{}.json".format(id)
    if not Path(missing_games_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchise/{}/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json".format(id)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(missing_games_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)

In [428]:
missing_games_ids = create_dataframe("missing_games_ids", missing_games_dir, True)
missing_games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,path
aliases,OK,1,0,1,1,1,None,1,raw_json\game_franchises\games\missing\3025-10...
api_detail_url,OK,1,0,1,1,1,https://www.giantbomb.com/api/franchise/3025-1...,1,raw_json\game_franchises\games\missing\3025-10...
characters,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-10...
concepts,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-10...
date_added,OK,1,0,1,1,1,2008-08-25 13:52:36,1,raw_json\game_franchises\games\missing\3025-10...
...,...,...,...,...,...,...,...,...,...
locations,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...
name,OK,1,0,1,1,1,Shaman King,1,raw_json\game_franchises\games\missing\3025-99...
objects,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...
people,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,raw_json\game_franchises\games\missing\3025-99...


We'll remove any rows that don't contain the game listings, rename the path column and extract the games ids

In [429]:
missing_games_ids = missing_games_ids.drop(missing_games_ids.index.difference(['games']))
missing_games_ids["path"] = missing_games_ids["path"].apply(lambda x : x[39:-5])
missing_games_ids = missing_games_ids.rename(columns = {"path":"franchise_id"})

<ipython-input-429-7780a6cf1ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_games_ids["path"] = missing_games_ids["path"].apply(lambda x : x[39:-5])


In [421]:
missing_games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version,franchise_id
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1000
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1001
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1002
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1003
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-1004
...,...,...,...,...,...,...,...,...,...
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-995
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-996
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-997
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1,3025-998


Finally, we'll combine our two dataframes to reintegrate the missing values into our original dataframe

In [430]:
games_ids = pd.concat([games_ids, missing_games_ids])
games_ids.franchise_id.count()

5524

And we can now see we have all the values we expected.

### Working with the combined dataframe

In [431]:
games_ids = drop_unnecessary_columns(games_ids)
games_ids

,results,franchise_id
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-1
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-10
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-100
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-101
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-102
...,...,...
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-995
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-996
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-997
games,[{'api_detail_url': 'https://www.giantbomb.com...,3025-998


In [ ]:
games_ids = games_ids.explode("results")
games_ids.count()

In [433]:
games_ids.count()

results         35117
franchise_id    35139
dtype: int64

We have some empty results so we'll drop those before moving on

In [435]:
games_ids = games_ids.dropna()
games_ids.count()

results         35117
franchise_id    35117
dtype: int64

We extract individual game names and ids. We can also extract the individual game ids so we can do further analysis on other games

In [436]:
games_ids[["game_id", "name", "api_url"]] = games_ids["results"].apply(lambda x: pd.Series([x["id"], x["name"], x["api_detail_url"]]))

D:\Programs\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [437]:
games_ids["game_id"].nunique()

29208

There's 289208 unique game ids which implies that around 6000 of our games are in multiple franchises. we can look at the ids of the games in the most franchises

In [438]:
games_ids["game_id"].value_counts().head()

6949     7
569      6
33847    6
73766    6
39509    5
Name: game_id, dtype: int64

# UNTESTED CODE FROM THIS POINT ON - WORK IN PROGRESS

In [439]:
games_ids

,results,franchise_id,game_id,name,api_url
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-1,544,Super Mario All-Stars & Super Mario World,https://www.giantbomb.com/api/game/3030-544/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-1,6649,Super Mario Sunshine,https://www.giantbomb.com/api/game/3030-6649/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-1,7314,Super Mario Bros. Deluxe,https://www.giantbomb.com/api/game/3030-7314/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-1,7358,Super Mario RPG: Legend of the Seven Stars,https://www.giantbomb.com/api/game/3030-7358/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-1,7406,Super Mario Bros. 2,https://www.giantbomb.com/api/game/3030-7406/
...,...,...,...,...,...
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-999,26577,Shaman King: Funbari Spirits,https://www.giantbomb.com/api/game/3030-26577/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-999,26575,"Shaman King: Legacy of the Spirits, Soaring Hawk",https://www.giantbomb.com/api/game/3030-26575/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-999,26581,Shaman King Chou Senjiryokketsu: Meramera/Funb...,https://www.giantbomb.com/api/game/3030-26581/
games,{'api_detail_url': 'https://www.giantbomb.com/...,3025-999,26580,Shaman King: Soul Fight!,https://www.giantbomb.com/api/game/3030-26580/
